In [27]:
import os, sys, importlib

import geopandas as gpd
import pandas as pd

import shapely.ops
from shapely.geometry import Point, LineString
from shapely import ops

from osgeo import ogr


In [2]:
baseFolder = os.path.join(os.getcwd(), "SampleData","Canals_sample_2")
#This points to the input lines file - points to sample data right now
inLines = os.path.join(baseFolder, 'Canals_sample_2.shp')
#Enter the folder where you want output created
unqID = "UID"
inL = gpd.read_file(inLines)
#inL = inL.to_crs({'init':'epsg:3857'}) #need to project to a CRS with metres 
inL.head()

,CHANNEL_NA,CHANNEL_TY,SHAPE_Leng,Head_X,Head_y,Tail_X,Tail_Y,LENGTH,Shape_Le_1,UID,geometry
0,2R/3R Minor,M,2366.669831,72.345250,30.151624,72.320696,30.151392,2366.669831,2366.669831,26005000,"LINESTRING (244300.021 3338563.505, 241933.537..."
1,6CR Disty,D,2057.445263,73.033955,30.636666,73.026671,30.654103,2057.445263,2057.445263,20000000,"LINESTRING (311583.662 3390984.321, 311325.528..."
2,6BR Disty,D,2708.487795,73.022003,30.633134,72.999000,30.645735,2708.487795,2708.487795,21000000,"LINESTRING (310431.169 3390612.871, 310376.282..."
3,7CR Disty,D,368.060039,72.581546,30.548341,72.581563,30.551659,368.060039,368.060039,34000000,"LINESTRING (268005.187 3382038.088, 268016.465..."
4,1 L Disty,D,31090.993270,73.615491,30.814278,73.615494,30.814281,31090.993270,31090.993270,04000000,"MULTILINESTRING ((367564.749 3409839.289, 3674..."


In [3]:
def generate_subset_line(allCoords, length):
    #Generate the 
    allPts = []
    for cIdx in range(0, len(allCoords)):
        cPoint = Point(allCoords[cIdx])
        allPts.append(cPoint)
        if cIdx > 0:
            curLine = LineString(allPts)
            if curLine.length > length:
                break
    return(curLine)

In [29]:
start = []
end = []
riverNameColumn = unqID
inL['startPer'] = 0
inL['endPer'] = 0
for idx, row in inL.iterrows():
    x = row['geometry']
    if x.geom_type == 'MultiLineString':
        x = ops.linemerge(x)
    x = ogr.CreateGeometryFromWkt(x.wkt)
    if x.GetGeometryType() == 2 or x.GetGeometryType() == 5:
        totalLength = x.Length()
        x.Segmentize(round(totalLength/100))    
        startLength = totalLength * 0.4
        endLength   = totalLength * 0.2
        allCoords = list(x.GetPoints())

        startLen = generate_subset_line(allCoords, startLength)
        allCoords.reverse()
        endLen   = generate_subset_line(allCoords, endLength)

        start.append([row[riverNameColumn], startLen])
        end.append([row[riverNameColumn], endLen])
        
        inL['startPer'].loc[idx] = startLen.length/totalLength
        inL['endPer'].loc[idx] = endLen.length/totalLength
        
        print("%s - %s - %s - %s" % (row[riverNameColumn], x.Length(), startLen.length, endLen.length))
    else:
        break
startDF = pd.DataFrame(start, columns=["NAME", "GEOM"])
endDF = pd.DataFrame(end,  columns=["NAME", "GEOM"])


26005000 - 2366.669831125254 - 956.2302347980929 - 478.11511739903466
20000000 - 2057.4452628933905 - 842.1621346999596 - 421.4315999874266
21000000 - 2708.4877945779704 - 1092.3617720710258 - 565.6441078773947
34000000 - 368.06003869097054 - 150.56805723844582 - 74.848558805527


TypeError: 'NoneType' object is not iterable

In [31]:
x = row['geometry']
if x.geom_type == 'MultiLineString':
    x = ops.linemerge(x)
for l in x:
    print(l)

LINESTRING (367564.7488000002 3409839.289100001, 367484.9203000003 3409913.160800001, 367484.7646000003 3409913.6525, 367484.1919 3409913.834899999, 367386.6156000001 3410004.130100001, 367142.8060999997 3410247.9396, 366856.9604000002 3410508.5636, 366705.6304000001 3410664.097200001, 366512.2642000001 3410840.649, 366407.1738999998 3410924.7212, 366276.8618999999 3411025.607899999, 366049.8668 3411080.254899999, 365843.8898 3411080.254899999, 365642.1163999997 3411097.0693, 365336.0933999997 3411109.680199999, 364995.6008000001 3411147.512700001, 364802.2346000001 3411155.9199, 364638.2937000003 3411151.7163, 364457.5383000001 3411004.5899, 364306.2083000001 3410861.666999999, 364058.1952 3410664.097200001, 363499.1146999998 3410189.089, 363259.5088 3409970.5012, 363133.4003999997 3409852.800000001, 362843.3510999996 3409583.7688, 362645.7812999999 3409419.8279, 362309.4923 3409104.557, 361931.1672 3408797.6932)
LINESTRING (374737.2922 3430420.9717, 374780.7346999999 3430375.0822, 37

In [5]:
from shapely.wkt import loads

startDF_geom =startDF['GEOM']
startDF = startDF.drop(['GEOM'], axis=1)
startGDF = gpd.GeoDataFrame(startDF, geometry=startDF_geom, crs=inL.crs)

endDF_geom =endDF['GEOM']
endDF = endDF.drop(['GEOM'], axis=1)
endGDF = gpd.GeoDataFrame(endDF, geometry=endDF_geom, crs=inL.crs)



In [7]:
'''startDF.to_csv(os.path.join(baseFolder,"canal_starts.csv"))
endDF.to_csv(os.path.join(baseFolder,"canal_ends.csv"))
inL.to_csv(os.path.join(baseFolder,"canals_with_length.csv"))
'''
startGDF.to_file(os.path.join(baseFolder,"canal_starts.shp"))
endGDF.to_file(os.path.join(baseFolder,"canal_ends.shp"))
inL.to_file(os.path.join(baseFolder,"canal_with_length.shp"))

# Buffering and clipping
For each start and end segment, the feature needs to be buffered, but the buffer needs to be limited to not cover anything covered by other canals

In [8]:
bufferDist = 1000 # metres
bufferSet = startDF # endDF

for bufferSet, outName in [[startDF, 'canal_start_buffered_clipped'],[endDF, 'canal_end_buffered_clipped']]:
    bufferSet['GEOM2'] = ''
    outFile = os.path.join(baseFolder,"%s.shp" % outName)
    for idx, row in bufferSet.iterrows():
        curName = row['NAME']
        curShape = row['geometry']        
        bufferShape = curShape.buffer(bufferDist,cap_style=2, join_style=2, mitre_limit=1.0)

        #Select canals that intersect current buffered shape
        intersecting_canals = inL[inL.intersects(bufferShape)]
        if intersecting_canals.shape[0] > 1:
            intersecting_canals = intersecting_canals[intersecting_canals[riverNameColumn] != curName]        
            for selCanal in intersecting_canals['geometry']:
                xx = shapely.ops.split(bufferShape, selCanal)
                maxArea = 0            
                selArea = None
                for x in xx:
                    if x.area > maxArea:
                        maxArea = x.area
                        selArea = x
                bufferShape = selArea
        bufferSet['GEOM2'].iloc[idx] = bufferShape
    bufferSet_geom = bufferSet['GEOM2']
    bufferSetGDF = bufferSet.drop('GEOM2', axis=1)
    try:
        bufferSetGDF = bufferSetGDF.drop('GEOM', axis=1)
    except:
        pass
    bufferSetGDF = gpd.GeoDataFrame(bufferSetGDF, geometry=bufferSet_geom, crs=inL.crs)
    bufferSetGDF.to_file(outFile)
    #bufferSet.to_csv(outFile)

C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
